# Testig Recursive Method for Level2 Creation

In [15]:
import __init__

In [40]:
code_a = """ADM1
ADM1H
ADM2
ADM2H
ADM3
ADM3H
ADM4
ADM4H
ADM5
ADM5H
ADMD
ADMDH
LTER
PCL
PCLD
PCLF
PCLH
PCLI
PCLIX
PCLS
PRSH
TERR
ZN
ZNB"""

code_a = code_a.split("\n")
print(code_a)

['ADM1', 'ADM1H', 'ADM2', 'ADM2H', 'ADM3', 'ADM3H', 'ADM4', 'ADM4H', 'ADM5', 'ADM5H', 'ADMD', 'ADMDH', 'LTER', 'PCL', 'PCLD', 'PCLF', 'PCLH', 'PCLI', 'PCLIX', 'PCLS', 'PRSH', 'TERR', 'ZN', 'ZNB']


In [59]:
def recursive_main_function(codes: dict, depth: int) -> dict:
    depth_dict = {}
    # depth = depth - 1
    d = 2
    for key, items in codes.items():
        for item in items:
            
            if len(item) > depth + d:
                if item[:depth+d] not in depth_dict:
                    depth_dict[item[:depth+d]] = []
                depth_dict[item[:depth+d]].append(item)
            else:
                depth_dict[item] = [item]
    return depth_dict

def recursive(codes, depth: int, start_depth: int=0) -> dict:
    if start_depth != depth - 1:
        codes = recursive_main_function({"data":codes}, start_depth)
        for key, value in codes.items():
            codes[key] = recursive(value, depth, start_depth + 1)
    return codes

In [61]:
recursive_main_function({"data":code_a}, 1)

{'ADM': ['ADM1',
  'ADM1H',
  'ADM2',
  'ADM2H',
  'ADM3',
  'ADM3H',
  'ADM4',
  'ADM4H',
  'ADM5',
  'ADM5H',
  'ADMD',
  'ADMDH'],
 'LTE': ['LTER'],
 'PCL': ['PCL', 'PCLD', 'PCLF', 'PCLH', 'PCLI', 'PCLIX', 'PCLS'],
 'PRS': ['PRSH'],
 'TER': ['TERR'],
 'ZN': ['ZN'],
 'ZNB': ['ZNB']}

In [64]:
# start depht is always 2 and here we only chose the end depht
recursive(code_a, depth=3)

{'AD': {'ADM': ['ADM1',
   'ADM1H',
   'ADM2',
   'ADM2H',
   'ADM3',
   'ADM3H',
   'ADM4',
   'ADM4H',
   'ADM5',
   'ADM5H',
   'ADMD',
   'ADMDH']},
 'LT': {'LTE': ['LTER']},
 'PC': {'PCL': ['PCL', 'PCLD', 'PCLF', 'PCLH', 'PCLI', 'PCLIX', 'PCLS']},
 'PR': {'PRS': ['PRSH']},
 'TE': {'TER': ['TERR']},
 'ZN': {'ZN': ['ZN'], 'ZNB': ['ZNB']}}

# Create Level2 Based on Feature Codes

In [60]:
from datahandler.datareader import DataReader

feature_codes = "../datasets/Geonames/raw/featureCodes_en.txt"
feature_codes_df = DataReader.load_csv(feature_codes, sep='\t', names=["feature-code", "name", "description"])
feature_codes_df = feature_codes_df.drop(feature_codes_df[feature_codes_df['feature-code'].isnull()].index)
feature_codes_df['level-1'] = feature_codes_df['feature-code'].apply(lambda X: str(X)[0])
feature_codes_df['level-3'] = feature_codes_df['feature-code'].apply(lambda X: str(X)[2:])

level2_mapper = {}
for group, group_df in feature_codes_df.groupby("level-1"):
    group_df_level2_taxonomy = recursive(group_df['level-3'].tolist(), depth=3)
    print(f"group {group}: number of depth-2 items:‌ {len(group_df_level2_taxonomy)}")
    for depth2_name, depth2_items in group_df_level2_taxonomy.items():
        for depth3_name, depth3_items in depth2_items.items():
                for depth3_item in depth3_items:
                    # level2_mapper[level2] = level1_name  
                    level2_mapper[depth3_item] = depth3_name

feature_codes_df['level-2'] = feature_codes_df['level-3'].map(level2_mapper)
level3_columns = feature_codes_df.pop("level-3")
feature_codes_df.insert(5, "level-3", level3_columns)

display(feature_codes_df.head(10))

print("size of feature codes is:", feature_codes_df.shape)

group A: number of depth-2 items:‌ 6
group H: number of depth-2 items:‌ 57
group L: number of depth-2 items:‌ 34
group P: number of depth-2 items:‌ 2
group R: number of depth-2 items:‌ 12
group S: number of depth-2 items:‌ 115
group T: number of depth-2 items:‌ 62
group U: number of depth-2 items:‌ 47
group V: number of depth-2 items:‌ 11


,feature-code,name,description,level-1,level-2,level-3
0,A.ADM1,first-order administrative division,a primary administrative division of a country...,A,ADM,ADM1
1,A.ADM1H,historical first-order administrative division,a former first-order administrative division,A,ADM,ADM1H
2,A.ADM2,second-order administrative division,a subdivision of a first-order administrative ...,A,ADM,ADM2
3,A.ADM2H,historical second-order administrative division,a former second-order administrative division,A,ADM,ADM2H
4,A.ADM3,third-order administrative division,a subdivision of a second-order administrative...,A,ADM,ADM3
5,A.ADM3H,historical third-order administrative division,a former third-order administrative division,A,ADM,ADM3H
6,A.ADM4,fourth-order administrative division,a subdivision of a third-order administrative ...,A,ADM,ADM4
7,A.ADM4H,historical fourth-order administrative division,a former fourth-order administrative division,A,ADM,ADM4H
8,A.ADM5,fifth-order administrative division,a subdivision of a fourth-order administrative...,A,ADM,ADM5
9,A.ADM5H,historical fifth-order administrative division,a former fifth-order administrative division,A,ADM,ADM5H


size of feature codes is: (680, 6)


In [62]:
feature_codes_df['level-1'].value_counts()

S    253
H    137
T     98
U     62
L     48
A     24
R     22
P     18
V     18
Name: level-1, dtype: int64

# Analyze which level for Recursive function is appropiate for BEING‌ Level2

In [63]:
gp_df = feature_codes_df.groupby("level-1")

In [66]:
###‌       DEPTH = 2
depth2_gps = {}
for gp, df in gp_df:
    depth2_gps[gp] = recursive(gp_df.get_group(gp)['level-3'].tolist(), depth=2)
    
for row in depth2_gps.keys():
    print(f"\t{row}", end=' ')
print()
for row in depth2_gps.keys():
    print(f"{row}", end="\t")
    for col in depth2_gps.keys():
        print(len(set(depth2_gps[row].keys()).intersection(depth2_gps[col].keys())), end="\t")
    print()

	A 	H 	L 	P 	R 	S 	T 	U 	V 
A	6	0	1	0	1	4	1	2	0	
H	0	57	4	1	3	26	16	15	2	
L	1	4	34	0	4	19	13	9	2	
P	0	1	0	2	1	2	0	0	0	
R	1	3	4	1	12	7	4	4	1	
S	4	26	19	2	7	115	30	23	6	
T	1	16	13	0	4	30	62	24	5	
U	2	15	9	0	4	23	24	47	3	
V	0	2	2	0	1	6	5	3	11	


In [68]:
###‌       DEPTH = 3
depth3_gps = {}
for gp, df in gp_df:
    # level1_gps[gp] = 
    depths = recursive(gp_df.get_group(gp)['level-3'].tolist(), depth=3)
    depth_dict = {}
    for key1, value1 in depths.items():
        for key2, value2 in value1.items():
            depth_dict[key2] = value2
    depth3_gps[gp] = depth_dict
    
for row in depth3_gps.keys():
    print(f"\t{row}", end=' ')
print()
for row in depth3_gps.keys():
    print(f"{row}", end="\t")
    for col in depth3_gps.keys():
        print(len(set(depth3_gps[row].keys()).intersection(depth3_gps[col].keys())), end="\t")
    print()

	A 	H 	L 	P 	R 	S 	T 	U 	V 
A	7	0	0	0	0	1	0	1	0	
H	0	71	0	0	1	6	4	5	0	
L	0	0	36	0	1	7	1	2	0	
P	0	0	0	2	0	0	0	0	0	
R	0	1	1	0	18	2	0	0	0	
S	1	6	7	0	2	165	1	2	2	
T	0	4	1	0	0	1	80	15	1	
U	1	5	2	0	0	2	15	62	0	
V	0	0	0	0	0	2	1	0	13	


In [69]:
###‌       DEPTH = 4
depth4_gps = {}
for gp, df in gp_df:
    # level1_gps[gp] = 
    depths = recursive(gp_df.get_group(gp)['level-3'].tolist(), depth=4)
    depth_dict = {}
    for key1, value1 in depths.items():
        for key2, value2 in value1.items():
            for key3, value3 in value2.items():
                depth_dict[key3] = value3
    depth4_gps[gp] = depth_dict
    
for row in depth4_gps.keys():
    print(f"\t{row}", end=' ')
print()
for row in depth4_gps.keys():
    print(f"{row}", end="\t")
    for col in depth4_gps.keys():
        print(len(set(depth4_gps[row].keys()).intersection(depth4_gps[col].keys())), end="\t")
    print()

	A 	H 	L 	P 	R 	S 	T 	U 	V 
A	17	0	0	0	0	0	0	0	0	
H	0	122	0	0	0	0	0	0	0	
L	0	0	48	0	0	0	0	0	0	
P	0	0	0	13	0	0	0	0	0	
R	0	0	0	0	21	0	0	0	0	
S	0	0	0	0	0	244	0	0	0	
T	0	0	0	0	0	0	97	0	0	
U	0	0	0	0	0	0	0	62	0	
V	0	0	0	0	0	0	0	0	16	
